# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,953.536497,0.247751,1234,953.784248,953.288746,1907.568497,1906.577491
6,788.563214,-0.903839,1234,787.659375,789.467054,1575.318750,1578.934107
7,913.933482,0.892795,1234,914.826277,913.040687,1829.652553,1826.081374
8,641.562296,0.818007,1234,642.380303,640.744289,1284.760606,1281.488578
9,591.279352,-0.784014,1234,590.495339,592.063366,1180.990677,1184.126732
10,153.528706,0.651708,1234,154.180414,152.876997,308.360828,305.753995
11,291.978787,0.367423,1234,292.346210,291.611363,584.692420,583.222727
12,623.591977,-0.036589,1234,623.555388,623.628566,1247.110777,1247.257131
13,465.430345,-0.815531,1234,464.614814,466.245876,929.229627,932.491752
14,946.223421,-0.675497,1234,945.547925,946.898918,1891.095849,1893.797836


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-953.784248,953.288746
6,-787.659375,789.467054
7,-914.826277,913.040687
8,-642.380303,640.744289
9,-590.495339,592.063366
10,-154.180414,152.876997
11,-292.346210,291.611363
12,-623.555388,623.628566
13,-464.614814,466.245876
14,-945.547925,946.898918


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-953.784248,953.288746
6,-787.659375,789.467054
7,-914.826277,913.040687
8,-642.380303,640.744289
9,-590.495339,592.063366
10,-154.180414,152.876997
11,-292.346210,291.611363
12,-623.555388,623.628566
13,-464.614814,466.245876
14,-945.547925,946.898918


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-953.78424838,  953.28874563],
       [-787.65937516,  789.46705365],
       [-914.8262767 ,  913.04068696],
       [-642.38030294,  640.74428914],
       [-590.49533861,  592.06336603],
       [-154.18041407,  152.87699744],
       [-292.34621018,  291.61136325],
       [-623.5553884 ,  623.62856568],
       [-464.61481352,  466.24587603],
       [-945.54792471,  946.89891776]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1905.568497,-1906.577491
6,1573.318750,-1578.934107
7,1827.652553,-1826.081374
8,1282.760606,-1281.488578
9,1178.990677,-1184.126732
10,306.360828,-305.753995
11,582.692420,-583.222727
12,1245.110777,-1247.257131
13,927.229627,-932.491752
14,1889.095849,-1893.797836


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
